<a href="https://colab.research.google.com/github/obie-udmz/watt/blob/main/Copy_of_WattPad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Document similarity and ranking

#### Import Library 
All the libraries needed for running this program is imported here

In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

#### Import dataset and GLoVe model file 

In [3]:
!git clone https://github.com/obie-udmz/watt.git

Cloning into 'watt'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [4]:
df = pd.read_table('watt/corpus_coding_test.tsv', sep='\t')

In [7]:
data = pd.DataFrame(df.head(50))

In [8]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50 non-null     int64 
 1   doc_name  50 non-null     object
dtypes: int64(1), object(1)
memory usage: 928.0+ bytes
None


In [ ]:
data.rename(columns={ data.columns[0]: "id", data.columns[1]: "doc_name"}, inplace = True)

In [ ]:
data.head()

In [ ]:
data[data['id']==44081]

In [ ]:
data['doc_name'][0]

### Branch 2

In [ ]:
gloveFile = "glove.6B.50d.txt"

def loadGloveModel(gloveFile):
    print ("word vectors extraction begings")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    word_embedding = {}
    for line in content:
        values = line.split()
        words = values[0]
        embedding = np.array([float(val) for val in values[1:]])
        word_embedding[words] = embedding
    print ("extraction of ",len(word_embedding),"word vectors completed!")
    return word_embedding

In [ ]:
 word_embeddings=loadGloveModel(gloveFile)

word vectors extraction begings
extraction of  400000 word vectors completed!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

sentences = []
for s in data['doc_name']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [ ]:
sentences[:5]

['I hear it all the time boo!',
 "Too bad I ain't got the patience for anyone else's problems!",
 'That got me laughing even through all the pain.',
 "It really wasn't that good.",
 'I hugged me best friend.']

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((50,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((50,))
  sentence_vectors.append(v)

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,50), sentence_vectors[j].reshape(1,50))[0,0]

In [ ]:


nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

She rolled her eyes at me, “Ok, I’ve got to go, he’s giving me the stink eye for taking his phone.” She laughed again and I could hear Rosie laughing through the line too.
I looked her over to make sure everything was ok with her, I hadn’t been here for five weeks because everything was a little crazy at the moment with Cameron, Rosie and my job.
That was always the first thing out of her mouth, followed by…..  As I walked out of the room I was tackled into a hug by my mom, I laughed as she squeezed the life out of me.
Someone touched my back, making me jump because I hadn’t heard anyone else come in.
Oh Nate, go home and get her and bring her back with you!” my mom begged, looking at me with the pleading eyes that she used on my dad all the time.
I looked back to my mom to see she was still looking a little shocked.
I’d made sure to drop into the conversation tonight a few things about my job but by the look on his face he knew what I did for a living anyway, and he was definitely sca

In [ ]:
# Creating a dataframe object from ranked_sentences
MS_top = pd.DataFrame(ranked_sentences, columns = ['Score' , 'Sentence'], index = None )
MS_top.head(10)

,Score,Sentence
0,0.001412,"She rolled her eyes at me, “Ok, I’ve got to go..."
1,0.001411,I looked her over to make sure everything was ...
2,0.001410,That was always the first thing out of her mou...
3,0.001409,"Someone touched my back, making me jump becaus..."
4,0.001408,"Oh Nate, go home and get her and bring her bac..."
5,0.001408,I looked back to my mom to see she was still l...
6,0.001407,I’d made sure to drop into the conversation to...
7,0.001407,She started laughing at something Rosie had sa...
8,0.001406,“You really need to find a girl to look after ...
9,0.001406,"Go home and get them both!” my mom cried, with..."


In [ ]:
def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaner_words = list(set([w for w in words if w not in stopword_set]))

    return cleaner_words

def word_vec(s1):
    import scipy
    vectr = np.mean([word_embeddings[words] for words in preprocess(s1)],axis=0)
    return vectr


In [ ]:
def find_similar(cos_matrix, vector, top_n = 5):
    cosine_similarities = cosine_similarity(vector, cos_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [ ]:
def collector():
  input_id = int(input('enter document id (must be numbers): '))
  input_name = input('enter document excerpt or title: ')
  print('input: {} \t {}'.format(input_id,input_name))
  return input_id, input_name

In [ ]:
input_id, input_name = collector()

enter document id: 12345
enter document excerpt or title: yessir!
input: 12345 	 yessir!


In [ ]:
output_search = []
output_score = []
output_id = []
output_result = []
for index, score in find_similar(sentence_vectors, test_vec.reshape(1, -1)):
        output_score.append(score)
        output_search.append(MS_top['Sentence'][index])
        output_id.append(data[data['doc_name'].str.contains(MS_top['Sentence'][index])]['id'].iloc[0])
        output_result.append(data[data['doc_name'].str.contains(MS_top['Sentence'][index])]['doc_name'].iloc[0])


In [ ]:
output_result

['I caught up with Lisa quickly and grabbed her, throwing her over my shoulder while she thrashed and screamed, laughing her ass off. Hannah was taunting me getting teasingly close so I snapped my hand out and clamped it around her wrist, making her squeal. I carried Lisa and dragged Hannah into the water and then threw Lisa in, laughing as she coughed and spluttered, wiping water out of her face. She grinned at me and grabbed my hand pulling me into the water too and before I knew what was happening the other two girls jumped on me too. I heard my mom shout back, still laughing. “You girls knew this would happen, you deal with the consequences!”  I burst out laughing. Well I guess that’s what I get for falling asleep in front of three Peters girls, I should have known better! “You’d better run!” I shouted, streaking after one of them at random. It turned out to be Lisa so I smiled and pushed myself after her, making her scream at my mom for help, “Mom! Make him leave me alone!”  The t

In [ ]:
print('{} \t {} : {} \t {} : {} \t {} : {} \n {} : {} \t {} : {} '.format(input_id,output_id[0],output_score[0],output_id[1],output_score[1],output_id[2],output_score[2],output_id[3],output_score[3],output_id[4],output_score[4]))

12345 	 54461 : 0.9786640443542408 	 90041 : 0.9781648819118495 	 54461 : 0.9778631714302628 
 90041 : 0.9707312147016227 	 48027 : 0.9679920714939757 


In [ ]:
see['id'].iloc[0]

54461